In [48]:
import numpy as np
from einops import rearrange, reduce
from scipy import ndimage

import torch
import torch.nn as nn

In [42]:
# 1. sum along axes
(board.sum(axis=1) > 0).astype(int)

array([1, 0])

In [43]:
board

array([[ 0,  1,  0],
       [-1,  1, -1]])

In [258]:
row = np.array([0,1,1,1,0,-1,-1,-1])
board = np.zeros((6,8))
board[0,:] = row

In [59]:
board

array([[ 0.,  1.,  1.,  1.,  0., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [92]:
board

array([[ 0.,  1.,  1.,  1.,  0., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
vf(board_t) = neural_network(board_t) = 1-alpha*vf(board_t) + alpha*(board_t+1 + reward_t+1)

In [102]:
random_feature_vectors

array([[[ 0,  1,  0, -1],
        [-1, -1,  0,  0],
        [-1,  1, -1, -1]],

       [[-1,  1,  1, -1],
        [ 0,  0,  1,  1],
        [-1,  0, -1, -1]]])

In [221]:
board

array([[ 0.,  1.,  1.,  1.,  0., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [218]:
nn.functional.conv1d?

In [260]:
x = 100 # some random big number which only works with zero
filters = [
    ([x,1,1,1,x], 3),# double-open triplet
    ([x,-1,-1,-1,x], 3),
    ([x,1,1,x,0], 2), # double-open twople
    ([x,-1,-1,x,0], 2),
    ([0,1,x,1,0], 2), # single-gap triplet
    ([0,-1,x,-1,0], 2),
    ([0,1,1,1,x], 3),# single-open triplet, right
    ([0,-1,-1,-1,x], 3),
    ([x,1,1,1,0], 3),# single-open triplet, left
    ([x,-1,-1,-1,0], 3)
]
filter_matrix = np.array([f[0] for f in filters])
filter_tensor = torch.Tensor(rearrange(filter_matrix, ('in kernel -> in 1 kernel')))
targets = np.array([f[1] for f in filters])
targets = rearrange(torch.Tensor(targets), 'f -> 1 f 1')
layer = nn.Conv1d(in_channels=1, out_channels=len(filters), bias=False, kernel_size=5)
board_tensor = nn.functional.pad(torch.Tensor(board.reshape(6,1,8)), (2,2), mode='constant', value=100)
result = nn.functional.conv1d(board_tensor, filter_tensor, bias=None, padding='valid')
features = (result == targets).any(axis=0).any(axis=1).detach().numpy().astype(int)

In [256]:
features

array([1, 0, 0, 0, 0, 0, 1, 0, 1, 1])

In [249]:
result.shape

torch.Size([6, 10, 4])

In [233]:
features

array([1, 1, 0, 0, 0, 0, 1, 1, 1, 1])

In [261]:
x = 100 # some random big number which only works with zero
filters = [
    ([x,1,1,1,x], 3),# double-open triplet
    ([x,1,1,x,0], 2), # double-open twople
    ([0,1,x,1,0], 2), # single-gap triplet
    ([0,1,1,1,x], 3),# single-open triplet, right
    ([x,1,1,1,0], 3),# single-open triplet, left
]
filter_matrix = np.array([f[0] for f in filters])
filter_tensor = torch.Tensor(rearrange(filter_matrix, ('in kernel -> in 1 kernel')))
targets = np.array([f[1] for f in filters])
targets = rearrange(torch.Tensor(targets), 'f -> 1 f 1')
layer = nn.Conv1d(in_channels=1, out_channels=5, bias=False, kernel_size=5)
board_tensor = nn.functional.pad(torch.Tensor(board.reshape(6,1,8)), (2,2), mode='constant', value=100)
result = nn.functional.conv1d(board_tensor, filter_tensor, bias=None, padding='valid')
features_me  = (result ==  targets).any(axis=0).any(axis=1).detach().numpy().astype(int)
features_him = (result == -targets).any(axis=0).any(axis=1).detach().numpy().astype(int)

In [276]:
## 2d filters:

filters2d = [
    ([
        [1,0,0,0],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1]        
    ], 4),
    ([
        [0,0,0,1],
        [0,0,1,0],
        [0,1,0,0],
        [1,0,0,1]        
    ], 4),
#     ([
#         [0,0,0,0],
#         [0,1,0,0],
#         [0,0,1,0],
#         [0,0,0,1]        
#     ], 3),
#     ([
#         [0,0,0,0],
#         [0,0,1,0],
#         [0,1,0,0],
#         [1,0,0,1]        
#     ], 3),
]
filter_matrix2d = np.array([f[0] for f in filters2d])
filter_tensor2d = torch.Tensor(rearrange(filter_matrix2d, ('in a b -> in 1 a b')))
targets2d = targets = np.array([f[1] for f in filters2d])
targets2d = rearrange(torch.Tensor(targets2d), 'f -> 1 f 1')
# layer2d = nn.Conv2d(in_channels=1, out_channels=len(filter_matrix2d), bias=False, kernel_size=4)
result = nn.functional.conv2d(board_tensor, filter_tensor2d, bias=None, padding='valid')


RuntimeError: Given groups=1, weight of size [2, 1, 4, 4], expected input[1, 6, 1, 12] to have 1 channels, but got 6 channels instead

In [279]:
board_tensor = nn.functional.pad(torch.Tensor(board.reshape(6,1,8)), (2,2), mode='constant', value=100)


In [282]:
nn.functional.pad?

In [281]:
board_tensor.shape

torch.Size([6, 1, 12])

In [277]:
board_tensor.shape

torch.Size([6, 1, 12])

In [ ]:
result = nn.functional.conv2d(board_tensor, filter_tensor, bias=None, padding='valid')


In [271]:
filter_matrix2d.shape

(2, 4, 4)

In [ ]:
filter_tensor = torch.Tensor(rearrange(filter_matrix, ('in kernel -> in 1 kernel')))
targets = np.array([f[1] for f in filters])
targets = rearrange(torch.Tensor(targets), 'f -> 1 f 1')
layer = nn.Conv1d(in_channels=1, out_channels=5, bias=False, kernel_size=5)
board_tensor = nn.functional.pad(torch.Tensor(board.reshape(6,1,8)), (2,2), mode='constant', value=100)
result = nn.functional.conv1d(board_tensor, filter_tensor, bias=None, padding='valid')
features_me  = (result ==  targets).any(axis=0).any(axis=1).detach().numpy().astype(int)
features_him = (result == -targets).any(axis=0).any(axis=1).detach().numpy().astype(int)

In [190]:
targets

tensor([[[3.],
         [3.],
         [2.],
         [2.],
         [2.],
         [2.],
         [3.],
         [3.],
         [3.],
         [3.]]])

array([1, 1, 0, 0, 0, 0, 1, 1, 1, 1])

In [188]:
result.any(axis=0)

tensor([[ True,  True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True]])

In [172]:
%%timeit
nn.functional.conv1d(torch.Tensor(board.reshape(6,1,8)), filter_tensor, bias=None, padding='same')


150 µs ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [170]:
nn.functional.conv1d?

In [158]:
filter_matrix.shape

(10, 5)

In [163]:
p.data.

RuntimeError: fill_ only supports 0-dimension value tensor but got tensor with 3 dimensions.

In [128]:
for p in layer.parameters():
    print(p)

Parameter containing:
tensor([[[-0.3208,  0.2761,  0.3775, -0.2660,  0.1800]],

        [[-0.0503, -0.2926,  0.1995,  0.3077,  0.4458]],

        [[ 0.3979, -0.4240,  0.1047,  0.0652,  0.0152]],

        [[ 0.0181, -0.1986,  0.3607,  0.2160, -0.1692]],

        [[ 0.3314,  0.3982,  0.2777,  0.3729,  0.0560]],

        [[-0.1033, -0.4256, -0.2859,  0.1324,  0.0803]],

        [[-0.2195, -0.2319, -0.0796,  0.0173, -0.3386]],

        [[-0.1479,  0.3164,  0.4013,  0.2575,  0.2271]],

        [[ 0.0406, -0.3638, -0.3731, -0.3903, -0.2841]],

        [[-0.2124, -0.1062, -0.3017,  0.3600, -0.0796]]], requires_grad=True)


In [110]:
filter_matrix

array([[100,   1,   1,   1, 100],
       [100,  -1,  -1,  -1, 100],
       [100,   1,   1, 100,   0],
       [100,  -1,  -1, 100,   0],
       [  0,   1, 100,   1,   0],
       [  0,  -1, 100,  -1,   0],
       [  0,   1,   1,   1, 100],
       [  0,  -1,  -1,  -1, 100],
       [100,   1,   1,   1,   0],
       [100,  -1,  -1,  -1,   0]])

In [97]:
def get_features(board):

    ## 1d filter and target value. 

    x = 100 # some random big number which only works with zero
    filters = [
        ([x,1,1,1,x], 3),# double-open triplet
        ([x,-1,-1,-1,x], 3),
        ([x,1,1,x,0], 2), # double-open twople
        ([x,-1,-1,x], 2),
        ([0,1,x,1,0], 2), # single-gap triplet
        ([0,-1,x,-1,0], 2),
        ([0,1,1,1,x], 3),# single-open triplet, right
        ([0,-1,-1,-1,x], 3),
        ([x,1,1,1,0], 3),# single-open triplet, left
        ([x,-1,-1,-1,0], 3)
    ]
    features = []
    for filter_, target in filters:
        features.append(int((ndimage.convolve1d(board, filter_, cval=13, mode='constant') == target).any()))
    bt = board.transpose()
    for filter_, target in filters:
        features.append(int((ndimage.convolve1d(bt, filter_, cval=13, mode='constant') == target).any()))
    return np.array(features)

In [98]:
%%timeit
get_features(board)

1.39 ms ± 97.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [100]:
def to_feature_vector(board):
    """
    TODO: Write this function to convert the state to a feature vector.

    We suggest you use functions in game_mechanics.py to make a handcrafted
     feature vector based on the state of the board.

    Feature vectors are covered in Tutorial 6 (don't need 4 or 5 to do 6!)

    Args:
        state: board state as a np array. 1's are your pieces. -1's your
                opponent's pieces & 0's are empty.

    Returns: the feature for this state, as designed by you.
    """
    mult = (rearrange(board, 'w h -> w h 1')*random_feature_vectors)
    random_features = (reduce(mult, 'w h f -> f', 'sum') > 0).astype(int)
    return random_features

In [105]:
%%timeit
to_feature_vector(board)

49.7 µs ± 4.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
ndimage.convolve1d(board, filter_, cval=13, mode='constant')

In [81]:
for filter_, target in filters:
    print(filter_, target)
    print((ndimage.convolve1d(board, filter_, cval=13, mode='constant') == target).any())

[100, 1, 1, 1, 100] 3
True
[100, -1, -1, -1, 100] 3
False
[100, 1, 1, 100] 2
False
[100, -1, -1, 100] 2
False
[1, 100, 1] 2
False
[-1, 100, -1] 2
False
[1, 1, 1, 100] 3
True
[-1, -1, -1, 100] 3
True
[100, 1, 1, 1] 3
True
[100, -1, -1, -1] 3
False


In [65]:
(ndimage.convolve1d(board, np.array([-100,1,1,1,-100])) == 3).any()

True

In [50]:
ndimage.find_objects?

In [53]:
## we are looking for [0,1,1,1,0].

In [54]:
target_subsequence = np.array([0,1,1,1,0])

In [55]:
target_subsequence in row

C:\Users\hristo\AppData\Local\Temp\ipykernel_19324\3968789267.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  target_subsequence in row


False

In [51]:
ndimage.find_objects(row, max_label=-2)

[(slice(1, 4, None),)]

In [ ]:
def max_cons(array, color=0):
    '''
    given 1d array, return the number of consequtive numbers of this color
    '''
    

In [ ]:
def features(board):
    ## horizontal dominance:
    h_dom = (board.sum(axis=1) > 0).astype(int)
    ## vertical dominance:
    v_dom = (board.sum(axis=1) > 0).astype(int)

In [17]:
# 2. number of chips of given color along given axis
(board == 1).sum(axis=1)

array([1, 1])

In [12]:
# same thing for diagonals
board.diagonal(offset=2)

array([0])

In [16]:
board.diagonal(axis1=1,axis2=0, offset=-1)

array([ 1, -1])

In [ ]:
# 3. consequtive chips of a color along rows


# 4. fuck all that, the search space is becoming as big as the original

In [ ]:
random_matrix = np.random.randint

In [104]:
random_feature_vectors = np.random.randint(-1,2,(6,8,15))

In [28]:
a = (rearrange(board, 'w h -> w h 1')*random_feature_vectors)

In [36]:
rf = (reduce(a, 'w h f -> f', 'sum') > 0).astype(int)

In [37]:
rf

array([1, 0, 1, 1])